In [ ]:
from google.colab import drive
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import matplotlib.pyplot as plt

drive.mount('/content/drive')

In [ ]:
#img=cv2.imread('/content/drive/MyDrive/IP/Images/rice.bmp')
#img=cv2.imread('/content/drive/MyDrive/IP/Images/lena256.jpg')
#img=cv2.imread('/content/drive/MyDrive/IP/Images/cell.bmp')
#img=cv2.imread('/content/drive/MyDrive/IP/Images/milkdrop.bmp')
img=cv2.imread('/content/drive/MyDrive/IP/Images/cell-bright.bmp')
print(img.shape)
gImg1=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#plt.imshow(gImg1,cmap='gray')
cv2_imshow(gImg1)

In [ ]:
hist=cv2.calcHist([gImg1],[0],None,[256],[0,255])
plt.plot(hist)

In [ ]:
gBin=np.zeros((gImg1.shape[0], gImg1.shape[1]), dtype=np.ubyte)

In [ ]:
Threshold=180
for h in range(gImg1.shape[0]):
    for w in range(gImg1.shape[1]):
        if gImg1[h,w]>Threshold:
            gBin[h,w]=0
        else:
            gBin[h,w]=255

cv2_imshow(gBin)

In [ ]:
T1=150
T2=0
T0=1

In [ ]:
def threshold_update(img, bImg):
    sum1=0
    count1=1
    sum2=0
    count2=1
    for h in range(img.shape[0]):
        for w in range(img.shape[1]):
            if(bImg[h,w]==0):
                sum1+=img[h,w]
                count1+=1
            else:
                sum2=img[h,w]
                count2+=1
        avg1=sum1/count1
        avg2=sum2/count2
        T=(avg1+avg2)/2
        return T

In [ ]:
for h in range(img.shape[0]):
    for w in range(img.shape[1]):
        if gImg1[h,w]>T1:
            gBin[h,w]=255
        else:
            gBin[h,w]=0

cv2_imshow(gBin)

In [ ]:
while True:
    T2=threshold_update(gImg1, gBin)
    print(T2)

    if(np.abs(T1-T2)<T0):
        break;
    else:
        T1=T2
        for h in range(img.shape[0]):
            for w in range(img.shape[1]):
                if gImg1[h,w]>T1:
                    gBin[h,w]=0
                else:
                    gBin[h,w]=255

print(T2)
cv2_imshow(gBin)

In [ ]:
N=10

dimh=np.int32(gImg1.shape[0]/N)
dimw=np.int32(gImg1.shape[1]/N)

dh_rest=np.int32(gImg1.shape[0]%N)
dw_rest=np.int32(gImg1.shape[1]%N)

meanImg=np.zeros((N,N))
outImg=np.zeros((gImg1.shape[0], gImg1.shape[1]), dtype=np.ubyte)


In [ ]:
def mean_function(Img, dimh, dimw, h, w):
    count=1
    sum=np.int64(0)
    avg=0

    for y in range(h,h+dimh):
        for x in range(w, w+dimw):
            sum=sum+Img[y,x]
            count+=1
    avg=np.int32(sum/count)
    return avg

In [ ]:
for h in range(0, gImg1.shape[0]-dh_rest,dimh):
    for w in range(0, gImg1.shape[1]-dw_rest, dimw):
         if(w+dimh < gImg1.shape[0] and w+dimw<gImg1.shape[1]):
              meanImg[np.int32(h//dimh),np.int32( w//dimw)]=mean_function(gImg1, dimh, dimw, h, w)

In [ ]:
for h in range(0,gImg1.shape[0]-dh_rest,dimh):
    for w in range(0,gImg1.shape[1]-dw_rest,dimw):
        if(gImg1[h,w]>=meanImg[np.int32(h//dimh),np.int32(w//dimw)]):
            outImg[h,w]=0
        else:
            outImg[h,w]=255

In [ ]:
print(meanImg)
cv2_imshow(outImg)